**FORWARD PROP ON FIRST TRAINING EXAMPLE TO MATCH THE RESULTS**

In [ ]:
%%writefile mnist_mlp.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <npy.hpp> // Include npy.hpp for loading .npy files

#define INPUT_SIZE 784
#define HIDDEN1_SIZE 512
#define HIDDEN2_SIZE 256
#define OUTPUT_SIZE 10

__device__ float relu(float x) {
    return fmaxf(0.0f, x);
}

__device__ float softmax(float* output, int idx, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        sum += expf(output[i]);
    }
    return expf(output[idx]) / sum;
}

__global__ void linear_layer_and_activation_relu(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = relu(z);
    }
}

__global__ void linear_layer_and_activation(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = z;
    }
}

__global__ void softmax_layer(float *input, float *output, int size) {
    float max_val = input[0];
    for (int i = 1; i < size; ++i) {
        if (input[i] > max_val) max_val = input[i];
    }

    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        output[i] = expf(input[i] - max_val);
        sum += output[i];
    }

    for (int i = 0; i < size; ++i) {
        output[i] /= sum;
    }
}

__global__ void print_probabilities(float *output, int size) {
    printf("Class probabilities:\n");
    for (int i = 0; i < size; ++i) {
        printf("Class %d: %f\n", i, output[i]);
    }
}

__global__ void get_predicted_class(float *output, int *predicted_class, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        int class_idx = 0;
        float max_val = output[0];
        for (int i = 1; i < output_size; ++i) {
            if (output[i] > max_val) {
                max_val = output[i];
                class_idx = i;
            }
        }
        *predicted_class = class_idx;
    }
}

__global__ void compute_cross_entropy_loss(float *output, double *labels, float *loss, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        // Find the true class index from the one-hot encoded label
        int true_class_idx = -1;
        for (int i = 0; i < size; ++i) {
            if (labels[i] == 1.0) {
                true_class_idx = i;
                break;
            }
        }

        if (true_class_idx != -1) {
            // Compute cross-entropy loss
            float prob = output[true_class_idx];
            if (prob > 0) {
                *loss = -logf(prob);
            } else {
                *loss = 0.0f;  // Handle case where probability is zero
            }
        } else {
            *loss = 0.0f;  // Handle the case where no valid class is found
        }
    }
}

int main() {
    const int input_size = INPUT_SIZE;
    const int hidden1_size = HIDDEN1_SIZE;
    const int hidden2_size = HIDDEN2_SIZE;
    const int output_size = OUTPUT_SIZE;

    // Allocate memory on the host
    std::vector<float> host_input;
    std::vector<float> host_hidden1_weights;
    std::vector<float> host_hidden1_biases;
    std::vector<float> host_hidden2_weights;
    std::vector<float> host_hidden2_biases;
    std::vector<float> host_output_weights;
    std::vector<float> host_output_biases;
    std::vector<double> host_labels;  // One-hot encoded labels

    std::vector<unsigned long> shape;

    // Load MNIST data
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", shape, host_input);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", shape, host_labels);

    // Load weights and biases
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", shape, host_hidden1_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_biases.npy", shape, host_hidden1_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_weights.npy", shape, host_hidden2_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_biases.npy", shape, host_hidden2_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_weights.npy", shape, host_output_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_biases.npy", shape, host_output_biases);

    // Allocate memory on the device
    float *d_input, *d_hidden1_output, *d_hidden2_output, *d_output_output;
    float *d_hidden1_weights, *d_hidden1_biases, *d_hidden2_weights, *d_hidden2_biases, *d_output_weights, *d_output_biases;
    double *d_labels;
    float *d_loss;
    int *d_predicted_class;

    cudaMalloc((void**)&d_input, input_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_weights, input_size * hidden1_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_biases, hidden1_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_output, hidden1_size * sizeof(float));

    cudaMalloc((void**)&d_hidden2_weights, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc((void**)&d_hidden2_biases, hidden2_size * sizeof(float));
    cudaMalloc((void**)&d_hidden2_output, hidden2_size * sizeof(float));

    cudaMalloc((void**)&d_output_weights, hidden2_size * output_size * sizeof(float));
    cudaMalloc((void**)&d_output_biases, output_size * sizeof(float));
    cudaMalloc((void**)&d_output_output, output_size * sizeof(float));

    cudaMalloc((void**)&d_labels, output_size * sizeof(double));  // One-hot encoded labels
    cudaMalloc((void**)&d_predicted_class, sizeof(int));
    cudaMalloc((void**)&d_loss, sizeof(float));

    // Copy data to device
    cudaMemcpy(d_input, host_input.data(), input_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_weights, host_hidden1_weights.data(), input_size * hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_biases, host_hidden1_biases.data(), hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_weights, host_hidden2_weights.data(), hidden1_size * hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_biases, host_hidden2_biases.data(), hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_weights, host_output_weights.data(), hidden2_size * output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_biases, host_output_biases.data(), output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_labels, host_labels.data(), output_size * sizeof(double), cudaMemcpyHostToDevice);

    // Launch kernels
    int block_size = 256;
    int grid_size = (hidden1_size + block_size - 1) / block_size;
    linear_layer_and_activation_relu<<<grid_size, block_size>>>(d_input, d_hidden1_weights, d_hidden1_biases, d_hidden1_output, input_size, hidden1_size);

    grid_size = (hidden2_size + block_size - 1) / block_size;
    linear_layer_and_activation_relu<<<grid_size, block_size>>>(d_hidden1_output, d_hidden2_weights, d_hidden2_biases, d_hidden2_output, hidden1_size, hidden2_size);

    grid_size = (output_size + block_size - 1) / block_size;
    linear_layer_and_activation<<<grid_size, block_size>>>(d_hidden2_output, d_output_weights, d_output_biases, d_output_output, hidden2_size, output_size);

    softmax_layer<<<1, 1>>>(d_output_output, d_output_output, output_size);
    print_probabilities<<<1, 1>>>(d_output_output, output_size);

    int predicted_class;
    cudaMemcpy(d_predicted_class, &predicted_class, sizeof(int), cudaMemcpyHostToDevice);
    get_predicted_class<<<1, 1>>>(d_output_output, d_predicted_class, output_size);

    float loss;
    compute_cross_entropy_loss<<<1, 1>>>(d_output_output, d_labels, d_loss, output_size);

    cudaMemcpy(&predicted_class, d_predicted_class, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&loss, d_loss, sizeof(float), cudaMemcpyDeviceToHost);

    std::cout << "Predicted class: " << predicted_class << std::endl;
    std::cout << "Cross-entropy loss: " << loss << std::endl;

    // Copy d_labels back to host to print them
    std::vector<double> host_labels_output(output_size);
    cudaMemcpy(host_labels_output.data(), d_labels, output_size * sizeof(double), cudaMemcpyDeviceToHost);

    std::cout << "Labels (one-hot encoded):" << std::endl;
    for (int i = 0; i < output_size; ++i) {
        std::cout << "Label " << i << ": " << host_labels_output[i] << std::endl;
    }

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_hidden1_weights);
    cudaFree(d_hidden1_biases);
    cudaFree(d_hidden1_output);
    cudaFree(d_hidden2_weights);
    cudaFree(d_hidden2_biases);
    cudaFree(d_hidden2_output);
    cudaFree(d_output_weights);
    cudaFree(d_output_biases);
    cudaFree(d_output_output);
    cudaFree(d_labels);
    cudaFree(d_loss);
    cudaFree(d_predicted_class);

    return 0;
}



Writing mnist_mlp.cu


In [ ]:
!nvcc -I/content/drive/MyDrive/Untitled_folder -o mnist_mlp mnist_mlp.cu


!./mnist_mlp


Class probabilities:
Class 0: 0.109313
Class 1: 0.058303
Class 2: 0.093253
Class 3: 0.070686
Class 4: 0.180806
Class 5: 0.126452
Class 6: 0.080382
Class 7: 0.094052
Class 8: 0.121958
Class 9: 0.064795
Predicted class: 4
Cross-entropy loss: 2.06789
Labels (one-hot encoded):
Label 0: 0
Label 1: 0
Label 2: 0
Label 3: 0
Label 4: 0
Label 5: 1
Label 6: 0
Label 7: 0
Label 8: 0
Label 9: 0


**FORWARD PROP ON FIRST FOUR TRAINING  EXAMPLES IN BATCH**

In [ ]:
%%writefile mnist_mlp_2.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <npy.hpp> // Include npy.hpp for loading .npy files

#define INPUT_SIZE 784
#define HIDDEN1_SIZE 512
#define HIDDEN2_SIZE 256
#define OUTPUT_SIZE 10
#define MINI_BATCH_SIZE 5

__device__ float relu(float x) {
    return fmaxf(0.0f, x);
}

__device__ float softmax(float* output, int idx, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        sum += expf(output[i]);
    }
    return expf(output[idx]) / sum;
}

__global__ void linear_layer_and_activation_relu(float *input, float *weights, float *biases, float *output, int input_size, int output_size, int batch_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int batch_idx = blockIdx.y;
    if (idx < output_size && batch_idx < batch_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[batch_idx * input_size + i];
        }
        output[batch_idx * output_size + idx] = relu(z);
    }
}

__global__ void linear_layer_and_activation(float *input, float *weights, float *biases, float *output, int input_size, int output_size, int batch_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int batch_idx = blockIdx.y;
    if (idx < output_size && batch_idx < batch_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[batch_idx * input_size + i];
        }
        output[batch_idx * output_size + idx] = z;
    }
}

__global__ void softmax_layer(float *input, float *output, int size, int batch_size) {
    int batch_idx = blockIdx.x;
    if (batch_idx < batch_size) {
        float max_val = input[batch_idx * size];
        for (int i = 1; i < size; ++i) {
            if (input[batch_idx * size + i] > max_val) max_val = input[batch_idx * size + i];
        }

        float sum = 0.0f;
        for (int i = 0; i < size; ++i) {
            output[batch_idx * size + i] = expf(input[batch_idx * size + i] - max_val);
            sum += output[batch_idx * size + i];
        }

        for (int i = 0; i < size; ++i) {
            output[batch_idx * size + i] /= sum;
        }
    }
}

__global__ void print_probabilities(float *output, int size, int batch_size) {
    int batch_idx = blockIdx.x;
    if (batch_idx < batch_size) {
        printf("Class probabilities for image %d:\n", batch_idx);
        for (int i = 0; i < size; ++i) {
            printf("Class %d: %f\n", i, output[batch_idx * size + i]);
        }
    }
}

__global__ void get_predicted_class(float *output, int *predicted_class, int output_size, int batch_size) {
    int batch_idx = blockIdx.x;
    if (batch_idx < batch_size) {
        int class_idx = 0;
        float max_val = output[batch_idx * output_size];
        for (int i = 1; i < output_size; ++i) {
            if (output[batch_idx * output_size + i] > max_val) {
                max_val = output[batch_idx * output_size + i];
                class_idx = i;
            }
        }
        predicted_class[batch_idx] = class_idx;
    }
}

__global__ void compute_cross_entropy_loss(float *output, double *labels, float *loss, int size, int batch_size) {
    int batch_idx = blockIdx.x;
    if (batch_idx < batch_size) {
        // Find the true class index from the one-hot encoded label
        int true_class_idx = -1;
        for (int i = 0; i < size; ++i) {
            if (labels[batch_idx * size + i] == 1.0) {
                true_class_idx = i;
                break;
            }
        }

        if (true_class_idx != -1) {
            // Compute cross-entropy loss
            float prob = output[batch_idx * size + true_class_idx];
            loss[batch_idx] = (prob > 0) ? -logf(prob) : 0.0f;
        } else {
            loss[batch_idx] = 0.0f;  // Handle the case where no valid class is found
        }
    }
}

int main() {
    const int input_size = INPUT_SIZE;
    const int hidden1_size = HIDDEN1_SIZE;
    const int hidden2_size = HIDDEN2_SIZE;
    const int output_size = OUTPUT_SIZE;
    const int batch_size = MINI_BATCH_SIZE;

    // Allocate memory on the host
    std::vector<float> host_input;
    std::vector<float> host_hidden1_weights;
    std::vector<float> host_hidden1_biases;
    std::vector<float> host_hidden2_weights;
    std::vector<float> host_hidden2_biases;
    std::vector<float> host_output_weights;
    std::vector<float> host_output_biases;
    std::vector<double> host_labels;  // One-hot encoded labels

    std::vector<unsigned long> shape;

    // Load MNIST data
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", shape, host_input);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", shape, host_labels);

    // Load weights and biases
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", shape, host_hidden1_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_biases.npy", shape, host_hidden1_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_weights.npy", shape, host_hidden2_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_biases.npy", shape, host_hidden2_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_weights.npy", shape, host_output_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_biases.npy", shape, host_output_biases);

    // Allocate memory on the device
    float *d_input, *d_hidden1_output, *d_hidden2_output, *d_output_output;
    float *d_hidden1_weights, *d_hidden1_biases, *d_hidden2_weights, *d_hidden2_biases, *d_output_weights, *d_output_biases;
    double *d_labels;
    float *d_loss;
    int *d_predicted_class;

    cudaMalloc((void**)&d_input, input_size * batch_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_weights, input_size * hidden1_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_biases, hidden1_size * sizeof(float));
    cudaMalloc((void**)&d_hidden1_output, hidden1_size * batch_size * sizeof(float));

    cudaMalloc((void**)&d_hidden2_weights, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc((void**)&d_hidden2_biases, hidden2_size * sizeof(float));
    cudaMalloc((void**)&d_hidden2_output, hidden2_size * batch_size * sizeof(float));

    cudaMalloc((void**)&d_output_weights, hidden2_size * output_size * sizeof(float));
    cudaMalloc((void**)&d_output_biases, output_size * sizeof(float));
    cudaMalloc((void**)&d_output_output, output_size * batch_size * sizeof(float));

    cudaMalloc((void**)&d_labels, output_size * batch_size * sizeof(double));  // One-hot encoded labels
    cudaMalloc((void**)&d_predicted_class, batch_size * sizeof(int));
    cudaMalloc((void**)&d_loss, batch_size * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_input, host_input.data(), input_size * batch_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_weights, host_hidden1_weights.data(), input_size * hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_biases, host_hidden1_biases.data(), hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_weights, host_hidden2_weights.data(), hidden1_size * hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_biases, host_hidden2_biases.data(), hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_weights, host_output_weights.data(), hidden2_size * output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_biases, host_output_biases.data(), output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_labels, host_labels.data(), output_size * batch_size * sizeof(double), cudaMemcpyHostToDevice);

    // Compute hidden1 layer
    dim3 threadsPerBlock(32);
    dim3 numBlocks((hidden1_size + threadsPerBlock.x - 1) / threadsPerBlock.x, batch_size);
    linear_layer_and_activation_relu<<<numBlocks, threadsPerBlock>>>(d_input, d_hidden1_weights, d_hidden1_biases, d_hidden1_output, input_size, hidden1_size, batch_size);

    // Compute hidden2 layer
    numBlocks = dim3((hidden2_size + threadsPerBlock.x - 1) / threadsPerBlock.x, batch_size);
    linear_layer_and_activation_relu<<<numBlocks, threadsPerBlock>>>(d_hidden1_output, d_hidden2_weights, d_hidden2_biases, d_hidden2_output, hidden1_size, hidden2_size, batch_size);

    // Compute output layer
    numBlocks = dim3((output_size + threadsPerBlock.x - 1) / threadsPerBlock.x, batch_size);
    linear_layer_and_activation<<<numBlocks, threadsPerBlock>>>(d_hidden2_output, d_output_weights, d_output_biases, d_output_output, hidden2_size, output_size, batch_size);

    // Apply softmax
    numBlocks = batch_size;
    softmax_layer<<<numBlocks, 1>>>(d_output_output, d_output_output, output_size, batch_size);

    // Compute cross-entropy loss
    compute_cross_entropy_loss<<<numBlocks, 1>>>(d_output_output, d_labels, d_loss, output_size, batch_size);

    // Get predicted class
    get_predicted_class<<<numBlocks, 1>>>(d_output_output, d_predicted_class, output_size, batch_size);

    // Copy results back to host
    std::vector<float> host_loss(batch_size);
    std::vector<int> host_predicted_class(batch_size);
    cudaMemcpy(host_loss.data(), d_loss, batch_size * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(host_predicted_class.data(), d_predicted_class, batch_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Print predictions and loss
    for (int i = 0; i < batch_size; ++i) {
        std::cout << "Image " << i << ":\n";
        std::cout << "Loss: " << host_loss[i] << "\n";
        std::cout << "Predicted class: " << host_predicted_class[i] << "\n";

        // Print actual class from one-hot encoded labels
        int actual_class = -1;
        for (int j = 0; j < output_size; ++j) {
            if (host_labels[i * output_size + j] == 1.0) {
                actual_class = j;
                break;
            }
        }
        std::cout << "Actual class: " << actual_class << "\n";
    }

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_hidden1_weights);
    cudaFree(d_hidden1_biases);
    cudaFree(d_hidden1_output);
    cudaFree(d_hidden2_weights);
    cudaFree(d_hidden2_biases);
    cudaFree(d_hidden2_output);
    cudaFree(d_output_weights);
    cudaFree(d_output_biases);
    cudaFree(d_output_output);
    cudaFree(d_labels);
    cudaFree(d_predicted_class);
    cudaFree(d_loss);

    return 0;
}


Overwriting mnist_mlp_2.cu


In [ ]:
!nvcc -I/content/drive/MyDrive/Untitled_folder -o mnist_mlp_2 mnist_mlp_2.cu
!./mnist_mlp_2

Image 0:
Loss: 2.06789
Predicted class: 4
Actual class: 5
Image 1:
Loss: 2.61446
Predicted class: 4
Actual class: 0
Image 2:
Loss: 2.03958
Predicted class: 4
Actual class: 4
Image 3:
Loss: 2.29785
Predicted class: 5
Actual class: 1
Image 4:
Loss: 2.65076
Predicted class: 4
Actual class: 9


**BACKWARD PROPAGATION ON SINGLE TRAINING EXAMPLE**

In [ ]:
%%writefile check.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <npy.hpp> // Include npy.hpp for loading .npy files

#define INPUT_SIZE 784
#define HIDDEN1_SIZE 512
#define HIDDEN2_SIZE 256
#define OUTPUT_SIZE 10

__device__ float relu(float x) {
    return fmaxf(0.0f, x);
}

__device__ float relu_derivative(float x) {
    return x > 0.0f ? 1.0f : 0.0f;
}

__device__ float softmax(float* output, int idx, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        sum += expf(output[i]);
    }
    return expf(output[idx]) / sum;
}

__global__ void linear_layer_and_activation_relu(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = relu(z);
    }
}

__global__ void linear_layer_and_activation(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = z;
    }
}

__global__ void softmax_layer(float *input, float *output, int size) {
    float max_val = input[0];
    for (int i = 1; i < size; ++i) {
        if (input[i] > max_val) max_val = input[i];
    }

    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        output[i] = expf(input[i] - max_val);
        sum += output[i];
    }

    for (int i = 0; i < size; ++i) {
        output[i] /= sum;
    }
}

__global__ void print_probabilities(float *output, int size) {
    printf("Class probabilities:\n");
    for (int i = 0; i < size; ++i) {
        printf("Class %d: %f\n", i, output[i]);
    }
}

__global__ void get_predicted_class(float *output, int *predicted_class, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        int class_idx = 0;
        float max_val = output[0];
        for (int i = 1; i < output_size; ++i) {
            if (output[i] > max_val) {
                max_val = output[i];
                class_idx = i;
            }
        }
        *predicted_class = class_idx;
    }
}

__global__ void compute_cross_entropy_loss(float *output, double *labels, float *loss, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        int true_class_idx = -1;
        for (int i = 0; i < size; ++i) {
            if (labels[i] == 1.0) {
                true_class_idx = i;
                break;
            }
        }
        if (true_class_idx != -1) {
            float prob = output[true_class_idx];
            if (prob > 0) {
                *loss = -logf(prob);
            } else {
                *loss = 0.0f;
            }
        } else {
            *loss = 0.0f;
        }
    }
}

__global__ void backpropagation_output(float *d_output, double *d_labels, float *d_dz3, float *d_dW3, float *d_db3, float *d_a2, int hidden2_size, int output_size) {
    int output_idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (output_idx < output_size) {
        // Compute error in the output layer (dz3)
        d_dz3[output_idx] = d_output[output_idx] - d_labels[output_idx];

        // Compute bias gradient (db3)
        d_db3[output_idx] = d_dz3[output_idx];

        // Compute weight gradients (dW3), iterate over the hidden2_size for the outer product
        for (int hidden_idx = 0; hidden_idx < hidden2_size; ++hidden_idx) {
            d_dW3[hidden_idx * output_size + output_idx] = d_a2[hidden_idx] * d_dz3[output_idx];
        }
    }
}

__global__ void backpropagation_hidden2(float *d_dz3, float *d_W3, float *d_a2, float *d_dz2, float *d_dW2, float *d_db2, int hidden1_size, int hidden2_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < hidden2_size) {
        float sum = 0.0f;
        for (int i = 0; i < OUTPUT_SIZE; ++i) {
            sum += d_dz3[i] * d_W3[idx * OUTPUT_SIZE + i];
        }
        d_dz2[idx] = relu_derivative(d_a2[idx]) * sum;

        // Accumulate gradients for weights
        d_dW2[idx] = 0.0f;
        for (int i = 0; i < hidden1_size; ++i) {
            d_dW2[idx * hidden1_size + i] = d_dz2[idx] * d_a2[i];
        }

        // Accumulate gradients for biases
        d_db2[idx] = d_dz2[idx];
    }
}



__global__ void backpropagation_hidden1(float *d_dz2, float *d_W2, float *d_x, float *d_dz1, float *d_dW1, float *d_db1, int input_size, int hidden1_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < hidden1_size) {
        float sum = 0.0f;
        for (int i = 0; i < HIDDEN2_SIZE; ++i) {
            sum += d_dz2[i] * d_W2[idx * HIDDEN2_SIZE + i];
        }
        d_dz1[idx] = relu_derivative(d_x[idx]) * sum;
        d_dW1[idx] = d_dz1[idx];
        d_db1[idx] = d_dz1[idx];
    }
}

__global__ void update_weights_biases(float *d_W1, float *d_b1, float *d_W2, float *d_b2, float *d_W3, float *d_b3,
                                      float *d_dW1, float *d_db1, float *d_dW2, float *d_db2, float *d_dW3, float *d_db3,
                                      float learning_rate, int input_size, int hidden1_size, int hidden2_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < input_size * hidden1_size) {
        d_W1[idx] -= learning_rate * d_dW1[idx];
    }
    if (idx < hidden1_size) {
        d_b1[idx] -= learning_rate * d_db1[idx];
    }
    if (idx < hidden1_size * hidden2_size) {
        d_W2[idx] -= learning_rate * d_dW2[idx];
    }
    if (idx < hidden2_size) {
        d_b2[idx] -= learning_rate * d_db2[idx];
    }
    if (idx < hidden2_size * output_size) {
        d_W3[idx] -= learning_rate * d_dW3[idx];
    }
    if (idx < output_size) {
        d_b3[idx] -= learning_rate * d_db3[idx];
    }
}

int main() {
    const int input_size = INPUT_SIZE;
    const int hidden1_size = HIDDEN1_SIZE;
    const int hidden2_size = HIDDEN2_SIZE;
    const int output_size = OUTPUT_SIZE;
    const float learning_rate = 0.01f;

    std::vector<float> host_input;
    std::vector<float> host_hidden1_weights;
    std::vector<float> host_hidden1_biases;
    std::vector<float> host_hidden2_weights;
    std::vector<float> host_hidden2_biases;
    std::vector<float> host_output_weights;
    std::vector<float> host_output_biases;
    std::vector<double> host_labels;

    std::vector<unsigned long> shape;

    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", shape, host_input);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", shape, host_labels);

    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", shape, host_hidden1_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_biases.npy", shape, host_hidden1_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_weights.npy", shape, host_hidden2_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_biases.npy", shape, host_hidden2_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_weights.npy", shape, host_output_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_biases.npy", shape, host_output_biases);

    float *d_input, *d_hidden1_weights, *d_hidden1_biases;
    float *d_hidden2_weights, *d_hidden2_biases, *d_output_weights, *d_output_biases;
    float *d_hidden1_output, *d_hidden2_output, *d_output, *d_softmax_output;
    float *d_dz1, *d_dz2, *d_dz3;
    float *d_dW1, *d_dW2, *d_dW3;
    float *d_db1, *d_db2, *d_db3;
    int *d_predicted_class;
    float *d_loss;
    double *d_labels;

    cudaMalloc(&d_input, input_size * sizeof(float));
    cudaMalloc(&d_hidden1_weights, input_size * hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden1_biases, hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden2_weights, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc(&d_hidden2_biases, hidden2_size * sizeof(float));
    cudaMalloc(&d_output_weights, hidden2_size * output_size * sizeof(float));
    cudaMalloc(&d_output_biases, output_size * sizeof(float));

    cudaMalloc(&d_hidden1_output, hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden2_output, hidden2_size * sizeof(float));
    cudaMalloc(&d_output, output_size * sizeof(float));
    cudaMalloc(&d_softmax_output, output_size * sizeof(float));

    cudaMalloc(&d_dz1, hidden1_size * sizeof(float));
    cudaMalloc(&d_dz2, hidden2_size * sizeof(float));
    cudaMalloc(&d_dz3, output_size * sizeof(float));

    cudaMalloc(&d_dW1, input_size * hidden1_size * sizeof(float));
    cudaMalloc(&d_dW2, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc(&d_dW3, hidden2_size * output_size * sizeof(float));

    cudaMalloc(&d_db1, hidden1_size * sizeof(float));
    cudaMalloc(&d_db2, hidden2_size * sizeof(float));
    cudaMalloc(&d_db3, output_size * sizeof(float));

    cudaMalloc(&d_predicted_class, sizeof(int));
    cudaMalloc(&d_loss, sizeof(float));
    cudaMalloc(&d_labels, output_size * sizeof(double));

    cudaMemcpy(d_input, host_input.data(), input_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_weights, host_hidden1_weights.data(), input_size * hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_biases, host_hidden1_biases.data(), hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_weights, host_hidden2_weights.data(), hidden1_size * hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_biases, host_hidden2_biases.data(), hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_weights, host_output_weights.data(), hidden2_size * output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_biases, host_output_biases.data(), output_size * sizeof(float), cudaMemcpyHostToDevice);

    cudaMemcpy(d_labels, host_labels.data(), output_size * sizeof(double), cudaMemcpyHostToDevice);

    int epochs = 35;

    for (int epoch = 0; epoch < epochs; ++epoch) {
        // Forward pass
        linear_layer_and_activation_relu<<<(hidden1_size + 255) / 256, 256>>>(d_input, d_hidden1_weights, d_hidden1_biases, d_hidden1_output, input_size, hidden1_size);
        linear_layer_and_activation_relu<<<(hidden2_size + 255) / 256, 256>>>(d_hidden1_output, d_hidden2_weights, d_hidden2_biases, d_hidden2_output, hidden1_size, hidden2_size);
        linear_layer_and_activation<<<(output_size + 255) / 256, 256>>>(d_hidden2_output, d_output_weights, d_output_biases, d_output, hidden2_size, output_size);
        softmax_layer<<<(output_size + 255) / 256, 256>>>(d_output, d_softmax_output, output_size);

        // Compute loss
        compute_cross_entropy_loss<<<1, 1>>>(d_softmax_output, d_labels, d_loss, output_size);
        float loss;
        cudaMemcpy(&loss, d_loss, sizeof(float), cudaMemcpyDeviceToHost);
        std::cout << "Epoch " << epoch + 1 << ", Loss: " << loss << std::endl;

        // Backpropagation
        backpropagation_output<<<(output_size + 255) / 256, 256>>>(d_softmax_output, d_labels, d_dz3, d_dW3, d_db3, d_hidden2_output, hidden2_size, output_size);
        backpropagation_hidden2<<<(hidden2_size + 255) / 256, 256>>>(d_dz3, d_output_weights, d_hidden2_output, d_dz2, d_dW2, d_db2, hidden1_size, hidden2_size);
        backpropagation_hidden1<<<(hidden1_size + 255) / 256, 256>>>(d_dz2, d_hidden2_weights, d_input, d_dz1, d_dW1, d_db1, input_size, hidden1_size);

        // Update weights and biases
        update_weights_biases<<<(input_size * hidden1_size + 255) / 256, 256>>>(d_hidden1_weights, d_hidden1_biases, d_hidden2_weights, d_hidden2_biases, d_output_weights, d_output_biases,
                                                                               d_dW1, d_db1, d_dW2, d_db2, d_dW3, d_db3, learning_rate,
                                                                               input_size, hidden1_size, hidden2_size, output_size);

        // Synchronize before printing
        cudaDeviceSynchronize();
     }

    // Free resources
    cudaFree(d_input);
    cudaFree(d_hidden1_weights);
    cudaFree(d_hidden1_biases);
    cudaFree(d_hidden2_weights);

    cudaFree(d_hidden2_biases);
    cudaFree(d_output_weights);
    cudaFree(d_output_biases);

    cudaFree(d_hidden1_output);
    cudaFree(d_hidden2_output);
    cudaFree(d_output);
    cudaFree(d_softmax_output);

    cudaFree(d_dz1);
    cudaFree(d_dz2);
    cudaFree(d_dz3);

    cudaFree(d_dW1);
    cudaFree(d_dW2);
    cudaFree(d_dW3);

    cudaFree(d_db1);
    cudaFree(d_db2);
    cudaFree(d_db3);

    cudaFree(d_predicted_class);
    cudaFree(d_loss);
    cudaFree(d_labels);

    return 0;
}


Overwriting check.cu


In [ ]:
!nvcc -I/content/drive/MyDrive/Untitled_folder -o check check.cu
!./check

Epoch 1, Loss: 2.06789
Epoch 2, Loss: 1.94808
Epoch 3, Loss: 1.83185
Epoch 4, Loss: 1.71941
Epoch 5, Loss: 1.61098
Epoch 6, Loss: 1.50676
Epoch 7, Loss: 1.40699
Epoch 8, Loss: 1.31186
Epoch 9, Loss: 1.22156
Epoch 10, Loss: 1.13621
Epoch 11, Loss: 1.05589
Epoch 12, Loss: 0.980652
Epoch 13, Loss: 0.910453
Epoch 14, Loss: 0.845214
Epoch 15, Loss: 0.784864
Epoch 16, Loss: 0.729481
Epoch 17, Loss: 0.678495
Epoch 18, Loss: 0.631655
Epoch 19, Loss: 0.588696
Epoch 20, Loss: 0.549342
Epoch 21, Loss: 0.513315
Epoch 22, Loss: 0.480346
Epoch 23, Loss: 0.450176
Epoch 24, Loss: 0.42256
Epoch 25, Loss: 0.397267
Epoch 26, Loss: 0.374084
Epoch 27, Loss: 0.352816
Epoch 28, Loss: 0.333283
Epoch 29, Loss: 0.315322
Epoch 30, Loss: 0.298756
Epoch 31, Loss: 0.283507
Epoch 32, Loss: 0.269427
Epoch 33, Loss: 0.256405
Epoch 34, Loss: 0.244345
Epoch 35, Loss: 0.233158


**BACKWARD PROPAGATION ON COMPLETE DATASET(Sample by sample)**

In [ ]:
%%writefile back.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <npy.hpp> // Include npy.hpp for loading .npy files

#define INPUT_SIZE 784
#define HIDDEN1_SIZE 512
#define HIDDEN2_SIZE 256
#define OUTPUT_SIZE 10

__device__ float relu(float x) {
    return fmaxf(0.0f, x);
}

__device__ float relu_derivative(float x) {
    return x > 0.0f ? 1.0f : 0.0f;
}

__device__ float softmax(float* output, int idx, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        sum += expf(output[i]);
    }
    return expf(output[idx]) / sum;
}

__global__ void linear_layer_and_activation_relu(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = relu(z);
    }
}

__global__ void linear_layer_and_activation(float *input, float *weights, float *biases, float *output, int input_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[i];
        }
        output[idx] = z;
    }
}

__global__ void softmax_layer(float *input, float *output, int size) {
    float max_val = input[0];
    for (int i = 1; i < size; ++i) {
        if (input[i] > max_val) max_val = input[i];
    }

    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        output[i] = expf(input[i] - max_val);
        sum += output[i];
    }

    for (int i = 0; i < size; ++i) {
        output[i] /= sum;
    }
}

__global__ void print_probabilities(float *output, int size) {
    printf("Class probabilities:\n");
    for (int i = 0; i < size; ++i) {
        printf("Class %d: %f\n", i, output[i]);
    }
}

__global__ void get_predicted_class(float *output, int *predicted_class, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        int class_idx = 0;
        float max_val = output[0];
        for (int i = 1; i < output_size; ++i) {
            if (output[i] > max_val) {
                max_val = output[i];
                class_idx = i;
            }
        }
        *predicted_class = class_idx;
    }
}

__global__ void compute_cross_entropy_loss(float *output, double *labels, float *loss, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx == 0) {
        int true_class_idx = -1;
        for (int i = 0; i < size; ++i) {
            if (labels[i] == 1.0) {
                true_class_idx = i;
                break;
            }
        }
        if (true_class_idx != -1) {
            float prob = output[true_class_idx];
            if (prob > 0) {
                *loss = -logf(prob);
            } else {
                *loss = 0.0f;
            }
        } else {
            *loss = 0.0f;
        }
    }
}

__global__ void backpropagation_output(float *d_output, double *d_labels, float *d_dz3, float *d_dW3, float *d_db3, float *d_a2, int hidden2_size, int output_size) {
    int output_idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (output_idx < output_size) {
        // Compute error in the output layer (dz3)
        d_dz3[output_idx] = d_output[output_idx] - d_labels[output_idx];

        // Compute bias gradient (db3)
        d_db3[output_idx] = d_dz3[output_idx];

        // Compute weight gradients (dW3), iterate over the hidden2_size for the outer product
        for (int hidden_idx = 0; hidden_idx < hidden2_size; ++hidden_idx) {
            d_dW3[hidden_idx * output_size + output_idx] = d_a2[hidden_idx] * d_dz3[output_idx];
        }
    }
}

__global__ void backpropagation_hidden2(float *d_dz3, float *d_W3, float *d_a2, float *d_dz2, float *d_dW2, float *d_db2, int hidden1_size, int hidden2_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < hidden2_size) {
        float sum = 0.0f;
        for (int i = 0; i < OUTPUT_SIZE; ++i) {
            sum += d_dz3[i] * d_W3[idx * OUTPUT_SIZE + i];
        }
        d_dz2[idx] = relu_derivative(d_a2[idx]) * sum;

        // Accumulate gradients for weights
        d_dW2[idx] = 0.0f;
        for (int i = 0; i < hidden1_size; ++i) {
            d_dW2[idx * hidden1_size + i] = d_dz2[idx] * d_a2[i];
        }

        // Accumulate gradients for biases
        d_db2[idx] = d_dz2[idx];
    }
}



__global__ void backpropagation_hidden1(float *d_dz2, float *d_W2, float *d_x, float *d_dz1, float *d_dW1, float *d_db1, int input_size, int hidden1_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < hidden1_size) {
        float sum = 0.0f;
        for (int i = 0; i < HIDDEN2_SIZE; ++i) {
            sum += d_dz2[i] * d_W2[idx * HIDDEN2_SIZE + i];
        }
        d_dz1[idx] = relu_derivative(d_x[idx]) * sum;
        d_dW1[idx] = d_dz1[idx];
        d_db1[idx] = d_dz1[idx];
    }
}


__global__ void update_weights_biases(float *d_W1, float *d_b1, float *d_W2, float *d_b2, float *d_W3, float *d_b3,
                                      float *d_dW1, float *d_db1, float *d_dW2, float *d_db2, float *d_dW3, float *d_db3,
                                      float learning_rate, int input_size, int hidden1_size, int hidden2_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < input_size * hidden1_size) {
        d_W1[idx] -= learning_rate * d_dW1[idx];
    }
    if (idx < hidden1_size) {
        d_b1[idx] -= learning_rate * d_db1[idx];
    }
    if (idx < hidden1_size * hidden2_size) {
        d_W2[idx] -= learning_rate * d_dW2[idx];
    }
    if (idx < hidden2_size) {
        d_b2[idx] -= learning_rate * d_db2[idx];
    }
    if (idx < hidden2_size * output_size) {
        d_W3[idx] -= learning_rate * d_dW3[idx];
    }
    if (idx < output_size) {
        d_b3[idx] -= learning_rate * d_db3[idx];
    }
}

int main() {
    const int input_size = INPUT_SIZE;
    const int hidden1_size = HIDDEN1_SIZE;
    const int hidden2_size = HIDDEN2_SIZE;
    const int output_size = OUTPUT_SIZE;
    const float learning_rate = 0.001f;

    std::vector<float> host_input;
    std::vector<float> host_hidden1_weights;
    std::vector<float> host_hidden1_biases;
    std::vector<float> host_hidden2_weights;
    std::vector<float> host_hidden2_biases;
    std::vector<float> host_output_weights;
    std::vector<float> host_output_biases;
    std::vector<double> host_labels;

    std::vector<unsigned long> shape;

    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", shape, host_input);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", shape, host_labels);

    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", shape, host_hidden1_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_biases.npy", shape, host_hidden1_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_weights.npy", shape, host_hidden2_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_biases.npy", shape, host_hidden2_biases);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_weights.npy", shape, host_output_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_biases.npy", shape, host_output_biases);

    // Allocate device memory
    float *d_input, *d_hidden1_weights, *d_hidden1_biases, *d_hidden2_weights, *d_hidden2_biases, *d_output_weights, *d_output_biases;
    double *d_labels;
    float *d_hidden1_output, *d_hidden2_output, *d_output;
    float *d_dW1, *d_db1, *d_dW2, *d_db2, *d_dW3, *d_db3;
    float *d_dz1, *d_dz2, *d_dz3;
    float *d_loss;
    int *d_predicted_class;
    int *predicted_class = new int;

    cudaMalloc(&d_input, input_size * sizeof(float));
    cudaMalloc(&d_hidden1_weights, input_size * hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden1_biases, hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden2_weights, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc(&d_hidden2_biases, hidden2_size * sizeof(float));
    cudaMalloc(&d_output_weights, hidden2_size * output_size * sizeof(float));
    cudaMalloc(&d_output_biases, output_size * sizeof(float));
    cudaMalloc(&d_labels, output_size * sizeof(double));

    cudaMalloc(&d_hidden1_output, hidden1_size * sizeof(float));
    cudaMalloc(&d_hidden2_output, hidden2_size * sizeof(float));
    cudaMalloc(&d_output, output_size * sizeof(float));

    cudaMalloc(&d_dW1, input_size * hidden1_size * sizeof(float));
    cudaMalloc(&d_db1, hidden1_size * sizeof(float));
    cudaMalloc(&d_dW2, hidden1_size * hidden2_size * sizeof(float));
    cudaMalloc(&d_db2, hidden2_size * sizeof(float));
    cudaMalloc(&d_dW3, hidden2_size * output_size * sizeof(float));
    cudaMalloc(&d_db3, output_size * sizeof(float));

    cudaMalloc(&d_dz1, hidden1_size * sizeof(float));
    cudaMalloc(&d_dz2, hidden2_size * sizeof(float));
    cudaMalloc(&d_dz3, output_size * sizeof(float));

    cudaMalloc(&d_loss, sizeof(float));
    cudaMalloc(&d_predicted_class, sizeof(int));

    // Copy data to device
    cudaMemcpy(d_input, host_input.data(), input_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_weights, host_hidden1_weights.data(), input_size * hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden1_biases, host_hidden1_biases.data(), hidden1_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_weights, host_hidden2_weights.data(), hidden1_size * hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hidden2_biases, host_hidden2_biases.data(), hidden2_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_weights, host_output_weights.data(), hidden2_size * output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_output_biases, host_output_biases.data(), output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_labels, host_labels.data(), output_size * sizeof(double), cudaMemcpyHostToDevice);

    int num_samples = 60000; // Number of samples to process one by one
    int num_epochs = 20; // Number of epochs
    for (int epoch = 0; epoch < num_epochs; ++epoch) {
    float total_loss = 0.0f; // Initialize total_loss at the start of each epoch
    for (int sample = 0; sample < num_samples; ++sample) {
        cudaMemcpy(d_input, host_input.data() + sample * input_size, input_size * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(d_labels, host_labels.data() + sample * output_size, output_size * sizeof(double), cudaMemcpyHostToDevice);

        // Forward pass
        linear_layer_and_activation_relu<<<(hidden1_size + 255) / 256, 256>>>(d_input, d_hidden1_weights, d_hidden1_biases, d_hidden1_output, input_size, hidden1_size);
        linear_layer_and_activation_relu<<<(hidden2_size + 255) / 256, 256>>>(d_hidden1_output, d_hidden2_weights, d_hidden2_biases, d_hidden2_output, hidden1_size, hidden2_size);
        linear_layer_and_activation<<<(output_size + 255) / 256, 256>>>(d_hidden2_output, d_output_weights, d_output_biases, d_output, hidden2_size, output_size);

        // Apply softmax
        softmax_layer<<<(output_size + 255) / 256, 256>>>(d_output, d_output, output_size);

        // Compute cross-entropy loss
        compute_cross_entropy_loss<<<1, 1>>>(d_output, d_labels, d_loss, output_size);

        float sample_loss;
        cudaMemcpy(&sample_loss, d_loss, sizeof(float), cudaMemcpyDeviceToHost);

        // Accumulate the loss for the current sample
        total_loss += sample_loss;

        // Backward pass
        get_predicted_class<<<1, 1>>>(d_output, d_predicted_class, output_size);
        cudaMemcpy(predicted_class, d_predicted_class, sizeof(int), cudaMemcpyDeviceToHost);

        backpropagation_output<<<(output_size + 255) / 256, 256>>>(d_output, d_labels, d_dz3, d_dW3, d_db3, d_hidden2_output, hidden2_size, output_size);
        backpropagation_hidden2<<<(hidden2_size + 255) / 256, 256>>>(d_dz3, d_output_weights, d_hidden2_output, d_dz2, d_dW2, d_db2, hidden1_size, hidden2_size);
        backpropagation_hidden1<<<(hidden1_size + 255) / 256, 256>>>(d_dz2, d_hidden2_weights, d_input, d_dz1, d_dW1, d_db1, input_size, hidden1_size);

        update_weights_biases<<<(input_size * hidden1_size + 255) / 256, 256>>>(
            d_hidden1_weights, d_hidden1_biases,
            d_hidden2_weights, d_hidden2_biases,
            d_output_weights, d_output_biases,
            d_dW1, d_db1, d_dW2, d_db2, d_dW3, d_db3,
            learning_rate, input_size, hidden1_size, hidden2_size, output_size
        );


    }
    float average_loss = total_loss / num_samples;
    std::cout << "Epoch " << epoch + 1 << " - Average Loss: " << average_loss << std::endl;
}

    // Free memory
    cudaFree(d_input);
    cudaFree(d_hidden1_weights);
    cudaFree(d_hidden1_biases);
    cudaFree(d_hidden2_weights);
    cudaFree(d_hidden2_biases);
    cudaFree(d_output_weights);
    cudaFree(d_output_biases);
    cudaFree(d_labels);
    cudaFree(d_hidden1_output);
    cudaFree(d_hidden2_output);
    cudaFree(d_output);
    cudaFree(d_dW1);
    cudaFree(d_db1);
    cudaFree(d_dW2);
    cudaFree(d_db2);
    cudaFree(d_dW3);
    cudaFree(d_db3);
    cudaFree(d_dz1);
    cudaFree(d_dz2);
    cudaFree(d_dz3);
    cudaFree(d_loss);
    cudaFree(d_predicted_class);
    delete[] predicted_class;
    return 0;
}

Writing back.cu


In [ ]:
!nvcc -I/content/drive/MyDrive/Untitled_folder -o back back.cu
!./back

Epoch 1 - Average Loss: 1.38764
Epoch 2 - Average Loss: 1.33207
Epoch 3 - Average Loss: 1.33125
Epoch 4 - Average Loss: 1.30591
Epoch 5 - Average Loss: 1.31623
Epoch 6 - Average Loss: 1.86744
Epoch 7 - Average Loss: 1.28567
Epoch 8 - Average Loss: 1.26964
Epoch 9 - Average Loss: 3.19138
Epoch 10 - Average Loss: 2.26431
Epoch 11 - Average Loss: 1.51665
Epoch 12 - Average Loss: 2.04775
Epoch 13 - Average Loss: 5.37715
Epoch 14 - Average Loss: 1.63338
Epoch 15 - Average Loss: 1.34183
Epoch 16 - Average Loss: 1.28342
Epoch 17 - Average Loss: 1.30111
Epoch 18 - Average Loss: 1.24131
Epoch 19 - Average Loss: 1.174
Epoch 20 - Average Loss: 1.93735


**BACKWARD PROPAGATION ON COMPLETE DATASET(In batches)**

In [ ]:
%%writefile back_2.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <npy.hpp> // Include npy.hpp for loading .npy files

#define INPUT_SIZE 784
#define HIDDEN1_SIZE 512
#define HIDDEN2_SIZE 256
#define OUTPUT_SIZE 10
#define BATCH_SIZE 128
#define LEARNING_RATE 0.001f

__device__ float relu(float x) {
    return fmaxf(0.0f, x);
}

__device__ float relu_derivative(float x) {
    return x > 0.0f ? 1.0f : 0.0f;
}

__device__ float softmax(float* output, int idx, int size) {
    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        sum += expf(output[i]);
    }
    return expf(output[idx]) / sum;
}

__global__ void linear_layer_and_activation_relu(float *input, float *weights, float *biases, float *output, int input_size, int output_size, int batch_size) {
    int batch_idx = blockIdx.y;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int batch_offset = batch_idx * input_size;

    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[batch_offset + i];
        }
        output[batch_idx * output_size + idx] = relu(z);
    }
}

__global__ void linear_layer_and_activation(float *input, float *weights, float *biases, float *output, int input_size, int output_size, int batch_size) {
    int batch_idx = blockIdx.y;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int batch_offset = batch_idx * input_size;

    if (idx < output_size) {
        float z = biases[idx];
        for (int i = 0; i < input_size; ++i) {
            z += weights[i * output_size + idx] * input[batch_offset + i];
        }
        output[batch_idx * output_size + idx] = z;
    }
}

__global__ void softmax_layer(float *input, float *output, int size, int batch_size) {
    int batch_idx = blockIdx.x;
    int idx = threadIdx.x;

    float *input_batch = input + batch_idx * size;
    float *output_batch = output + batch_idx * size;

    float max_val = input_batch[0];
    for (int i = 1; i < size; ++i) {
        if (input_batch[i] > max_val) max_val = input_batch[i];
    }

    float sum = 0.0f;
    for (int i = 0; i < size; ++i) {
        output_batch[i] = expf(input_batch[i] - max_val);
        sum += output_batch[i];
    }

    for (int i = 0; i < size; ++i) {
        output_batch[i] /= sum;
    }
}

__global__ void get_predicted_class(float *output, int *predicted_class, int output_size, int batch_size) {
    int batch_idx = blockIdx.x;
    int idx = threadIdx.x;

    float *output_batch = output + batch_idx * output_size;

    if (idx == 0) {
        int class_idx = 0;
        float max_val = output_batch[0];
        for (int i = 1; i < output_size; ++i) {
            if (output_batch[i] > max_val) {
                max_val = output_batch[i];
                class_idx = i;
            }
        }
        predicted_class[batch_idx] = class_idx;
    }
}

__global__ void compute_cross_entropy_loss(float *output, double *labels, float *loss, int size, int batch_size) {
    int batch_idx = blockIdx.x;
    int idx = threadIdx.x;

    float *output_batch = output + batch_idx * size;
    double *labels_batch = labels + batch_idx * size;

    if (idx == 0) {
        int true_class_idx = -1;
        for (int i = 0; i < size; ++i) {
            if (labels_batch[i] == 1.0) {
                true_class_idx = i;
                break;
            }
        }
        if (true_class_idx != -1) {
            float prob = output_batch[true_class_idx];
            if (prob > 0) {
                *loss = -logf(prob);
            } else {
                *loss = 0.0f;
            }
        } else {
            *loss = 0.0f;
        }
    }
}

__global__ void backpropagation_output(float *d_output, double *d_labels, float *d_dz3, float *d_dW3, float *d_db3, float *d_a2, int hidden2_size, int output_size, int batch_size) {
    int batch_idx = blockIdx.y;
    int output_idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (output_idx < output_size) {
        d_dz3[batch_idx * output_size + output_idx] = d_output[batch_idx * output_size + output_idx] - d_labels[batch_idx * output_size + output_idx];
        d_db3[output_idx] = d_dz3[batch_idx * output_size + output_idx];

        for (int hidden_idx = 0; hidden_idx < hidden2_size; ++hidden_idx) {
            d_dW3[hidden_idx * output_size + output_idx] += d_a2[batch_idx * hidden2_size + hidden_idx] * d_dz3[batch_idx * output_size + output_idx];
        }
    }
}

__global__ void backpropagation_hidden2(float *d_dz3, float *d_W3, float *d_a2, float *d_dz2, float *d_dW2, float *d_db2, int hidden1_size, int hidden2_size, int batch_size) {
    int batch_idx = blockIdx.y;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < hidden2_size) {
        float sum = 0.0f;
        for (int i = 0; i < OUTPUT_SIZE; ++i) {
            sum += d_dz3[batch_idx * OUTPUT_SIZE + i] * d_W3[idx * OUTPUT_SIZE + i];
        }
        d_dz2[batch_idx * hidden2_size + idx] = relu_derivative(d_a2[batch_idx * hidden2_size + idx]) * sum;

        for (int i = 0; i < hidden1_size; ++i) {
            d_dW2[idx * hidden1_size + i] += d_dz2[batch_idx * hidden2_size + idx] * d_a2[batch_idx * hidden1_size + i];
        }
        d_db2[idx] += d_dz2[batch_idx * hidden2_size + idx];
    }
}

__global__ void backpropagation_hidden1(float *d_dz2, float *d_W2, float *d_x, float *d_dz1, float *d_dW1, float *d_db1, int input_size, int hidden1_size, int batch_size) {
    int batch_idx = blockIdx.y;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < hidden1_size) {
        float sum = 0.0f;
        for (int i = 0; i < HIDDEN2_SIZE; ++i) {
            sum += d_dz2[batch_idx * HIDDEN2_SIZE + i] * d_W2[idx * HIDDEN2_SIZE + i];
        }
        d_dz1[batch_idx * hidden1_size + idx] = relu_derivative(d_x[batch_idx * input_size + idx]) * sum;
        d_dW1[idx] += d_dz1[batch_idx * hidden1_size + idx] * d_x[batch_idx * input_size + idx];
        d_db1[idx] += d_dz1[batch_idx * hidden1_size + idx];
    }
}

__global__ void update_weights_biases(float *d_W1, float *d_b1, float *d_W2, float *d_b2, float *d_W3, float *d_b3,
                                      float *d_dW1, float *d_db1, float *d_dW2, float *d_db2, float *d_dW3, float *d_db3,
                                      float learning_rate, int input_size, int hidden1_size, int hidden2_size, int output_size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < input_size * hidden1_size) {
        d_W1[idx] -= learning_rate * d_dW1[idx];
    } else if (idx < input_size * hidden1_size + hidden1_size) {
        d_b1[idx - input_size * hidden1_size] -= learning_rate * d_db1[idx - input_size * hidden1_size];
    } else if (idx < input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size) {
        d_W2[idx - (input_size * hidden1_size + hidden1_size)] -= learning_rate * d_dW2[idx - (input_size * hidden1_size + hidden1_size)];
    } else if (idx < input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size) {
        d_b2[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size)] -= learning_rate * d_db2[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size)];
    } else if (idx < input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size * output_size) {
        d_W3[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size)] -= learning_rate * d_dW3[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size)];
    } else {
        d_b3[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size * output_size)] -= learning_rate * d_db3[idx - (input_size * hidden1_size + hidden1_size + hidden1_size * hidden2_size + hidden2_size * output_size)];
    }
}

int main() {
    // Load dataset
    std::vector<float> train_images;
    std::vector<float> train_labels;

    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", {BATCH_SIZE, INPUT_SIZE}, train_images);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", {BATCH_SIZE, OUTPUT_SIZE}, train_labels);

    // Allocate and copy data to device
    float *d_input, *d_output;
    cudaMalloc(&d_input, BATCH_SIZE * INPUT_SIZE * sizeof(float));
    cudaMalloc(&d_output, BATCH_SIZE * OUTPUT_SIZE * sizeof(float));

    // Load weights and biases from .npy files
    float *d_weights1, *d_bias1;
    float *d_weights2, *d_bias2;
    float *d_weights3, *d_bias3;

    cudaMalloc(&d_weights1, INPUT_SIZE * HIDDEN1_SIZE * sizeof(float));
    cudaMalloc(&d_bias1, HIDDEN1_SIZE * sizeof(float));
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", {INPUT_SIZE, HIDDEN1_SIZE}, host_hidden1_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_biases.npy", {HIDDEN1_SIZE}, host_hidden1_biases);
    cudaMemcpy(d_weights1, host_hidden1_weights.data(), INPUT_SIZE * HIDDEN1_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bias1, host_hidden1_biases.data(), HIDDEN1_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    // Repeat for hidden2 and output layers
    cudaMalloc(&d_weights2, HIDDEN1_SIZE * HIDDEN2_SIZE * sizeof(float));
    cudaMalloc(&d_bias2, HIDDEN2_SIZE * sizeof(float));
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_weights.npy", {HIDDEN1_SIZE, HIDDEN2_SIZE}, host_hidden2_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden2_biases.npy", {HIDDEN2_SIZE}, host_hidden2_biases);
    cudaMemcpy(d_weights2, host_hidden2_weights.data(), HIDDEN1_SIZE * HIDDEN2_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bias2, host_hidden2_biases.data(), HIDDEN2_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    cudaMalloc(&d_weights3, HIDDEN2_SIZE * OUTPUT_SIZE * sizeof(float));
    cudaMalloc(&d_bias3, OUTPUT_SIZE * sizeof(float));
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_weights.npy", {HIDDEN2_SIZE, OUTPUT_SIZE}, host_output_weights);
    npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/output_biases.npy", {OUTPUT_SIZE}, host_output_biases);
    cudaMemcpy(d_weights3, host_output_weights.data(), HIDDEN2_SIZE * OUTPUT_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bias3, host_output_biases.data(), OUTPUT_SIZE * sizeof(float), cudaMemcpyHostToDevice);

    // Define other necessary variables and allocate memory for gradients
    float *d_dW1, *d_dW2, *d_dW3;
    float *d_db1, *d_db2, *d_db3;
    cudaMalloc(&d_dW1, INPUT_SIZE * HIDDEN1_SIZE * sizeof(float));
    cudaMalloc(&d_dW2, HIDDEN1_SIZE * HIDDEN2_SIZE * sizeof(float));
    cudaMalloc(&d_dW3, HIDDEN2_SIZE * OUTPUT_SIZE * sizeof(float));
    cudaMalloc(&d_db1, HIDDEN1_SIZE * sizeof(float));
    cudaMalloc(&d_db2, HIDDEN2_SIZE * sizeof(float));
    cudaMalloc(&d_db3, OUTPUT_SIZE * sizeof(float));

    // Training loop
    for (int epoch = 0; epoch < NUM_EPOCHS; ++epoch) {
        for (int batch = 0; batch < NUM_BATCHES; ++batch) {
            // Load data into d_input
            cudaMemcpy(d_input, train_images.data() + batch * BATCH_SIZE * INPUT_SIZE, BATCH_SIZE * INPUT_SIZE * sizeof(float), cudaMemcpyHostToDevice);

            // Forward pass
            dim3 threadsPerBlock(256);
            dim3 numBlocks1((HIDDEN1_SIZE + threadsPerBlock.x - 1) / threadsPerBlock.x, BATCH_SIZE);
            linear_layer_and_activation_relu<<<numBlocks1, threadsPerBlock>>>(d_input, d_weights1, d_bias1, d_a1, INPUT_SIZE, HIDDEN1_SIZE, BATCH_SIZE);

            dim3 numBlocks2((HIDDEN2_SIZE + threadsPerBlock.x - 1) / threadsPerBlock.x, BATCH_SIZE);
            linear_layer_and_activation_relu<<<numBlocks2, threadsPerBlock>>>(d_a1, d_weights2, d_bias2, d_a2, HIDDEN1_SIZE, HIDDEN2_SIZE, BATCH_SIZE);

            dim3 numBlocks3((OUTPUT_SIZE + threadsPerBlock.x - 1) / threadsPerBlock.x, BATCH_SIZE);
            linear_layer_and_activation<<<numBlocks3, threadsPerBlock>>>(d_a2, d_weights3, d_bias3, d_output, HIDDEN2_SIZE, OUTPUT_SIZE, BATCH_SIZE);

            // Softmax
            dim3 numBlocksSoftmax(BATCH_SIZE);
            softmax_layer<<<numBlocksSoftmax, OUTPUT_SIZE>>>(d_output, d_output, OUTPUT_SIZE, BATCH_SIZE);

            // Compute loss
            double *d_labels;
            cudaMalloc(&d_labels, BATCH_SIZE * OUTPUT_SIZE * sizeof(double));
            cudaMemcpy(d_labels, train_labels.data() + batch * BATCH_SIZE * OUTPUT_SIZE, BATCH_SIZE * OUTPUT_SIZE * sizeof(double), cudaMemcpyHostToDevice);

            float *d_loss;
            cudaMalloc(&d_loss, sizeof(float));
            compute_cross_entropy_loss<<<BATCH_SIZE, 1>>>(d_output, d_labels, d_loss, OUTPUT_SIZE, BATCH_SIZE);

            // Backpropagation
            // Add your backward pass kernels here
            // Example:
            backpropagation_output<<<numBlocks3, threadsPerBlock>>>(d_output, d_labels, d_dz3, d_dW3, d_db3, d_a2, HIDDEN2_SIZE, OUTPUT_SIZE, BATCH_SIZE);
            backpropagation_hidden2<<<numBlocks2, threadsPerBlock>>>(d_dz3, d_weights3, d_a2, d_dz2, d_dW2, d_db2, HIDDEN1_SIZE, HIDDEN2_SIZE, BATCH_SIZE);
            backpropagation_hidden1<<<numBlocks1, threadsPerBlock>>>(d_dz2, d_weights2, d_input, d_dz1, d_dW1, d_db1, INPUT_SIZE, HIDDEN1_SIZE, BATCH_SIZE);

            // Update weights and biases
            int numWeights = INPUT_SIZE * HIDDEN1_SIZE + HIDDEN1_SIZE + HIDDEN1_SIZE * HIDDEN2_SIZE + HIDDEN2_SIZE + HIDDEN2_SIZE * OUTPUT_SIZE + OUTPUT_SIZE;
            update_weights_biases<<<(numWeights + 255) / 256, 256>>>(d_weights1, d_bias1, d_weights2, d_bias2, d_weights3, d_bias3, d_dW1, d_db1, d_dW2, d_db2, d_dW3, d_db3, LEARNING_RATE, INPUT_SIZE, HIDDEN1_SIZE, HIDDEN2_SIZE, OUTPUT_SIZE);

            // Free d_labels and d_loss
            cudaFree(d_labels);
            cudaFree(d_loss);
        }
    }

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_weights1);
    cudaFree(d_weights2);
    cudaFree(d_weights3);
    cudaFree(d_bias1);
    cudaFree(d_bias2);
    cudaFree(d_bias3);
    cudaFree(d_dW1);
    cudaFree(d_dW2);
    cudaFree(d_dW3);
    cudaFree(d_db1);
    cudaFree(d_db2);
    cudaFree(d_db3);

    return 0;
}


Overwriting back_2.cu


In [ ]:
!nvcc -I/content/drive/MyDrive/Untitled_folder -o back_2 back_2.cu
!./back_2

back_2.cu(200): error: no instance of overloaded function "npy::LoadArrayFromNumpy" matches the argument list
            argument types are: (const char [51], {...}, std::vector<float, std::allocator<float>>)
      npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/x_train.npy", {128, 784}, train_images);
      ^

back_2.cu(201): error: no instance of overloaded function "npy::LoadArrayFromNumpy" matches the argument list
            argument types are: (const char [51], {...}, std::vector<float, std::allocator<float>>)
      npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/y_train.npy", {128, 10}, train_labels);
      ^

back_2.cu(215): error: identifier "host_hidden1_weights" is undefined
      npy::LoadArrayFromNumpy("/content/drive/MyDrive/Untitled_folder/hidden1_weights.npy", {784, 512}, host_hidden1_weights);
                                                                                                        ^

back_2.cu(215): error: no instance of

In [ ]:
from google.colab import drive
drive.mount('/content/drive')